In [99]:
%matplotlib inline

import io, os, sys, types

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
from numpy.linalg import eig, inv

import math

from scipy import interpolate, spatial, stats

import seaborn as sns

import skimage.io as skiIo
from skimage import exposure, img_as_float, filters

from sklearn import linear_model
from sklearn import metrics

import pickle


In [100]:
""" ============== path settings =============="""

' ============== path settings =============='

In [101]:
global isMac
isMac = False

if(isMac):
    supPrefix = '/Users/lily/Lily/Academic/AW_Lab/data/fate_switching_gfp_rfp_old/'
    dataPrefix = os.path.join(supPrefix, 'Data')
    figOutPrefix = os.path.join(supPrefix, 'Output', 'FigureOutput_v0422')
    dataOutPrefix = os.path.join(supPrefix, 'Output', 'DataOutput_v0422')
#     figOutPrefix = '/Users/lily/Lily/Academic/AW_Lab/Projects/NSP/Codes/python_v0415/Materials_trial/Output/FigureOutput'
#     dataOutPrefix = '/Users/lily/Lily/Academic/AW_Lab/Projects/NSP/Codes/python_v0415/Materials_trial/Output/DataOutput'
else:
    supPrefix = 'Z:\\lab\\Projects\\NSP\\Data_Analysis\\Experiments\\Fate_Switching\\GFP_RFP\\Summaries\\ver2018'
    dataPrefix = dataPrefix = os.path.join(supPrefix, 'Data')
    figOutPrefix = figOutPrefix = os.path.join(supPrefix, 'FigureOutput_v0422')
    dataOutPrefix = dataOutPrefix = os.path.join(supPrefix, 'DataOutput_v0422')

In [102]:
### file parameters
# file folders
imageFolder = 'Images'
ROIFolder = 'ROIs'
summaryFolder = 'Summaries'
dataFolder = 'DataOutput'

# file name
summaryName = 'Control_s3r1_summary.csv'

In [103]:
""" ============== Golbal variables and parameters =============="""

' ============== Golbal variables and parameters =============='

In [104]:
### global variables
global ColorCode, targetIndexMatch, bins, channel_mapping
bins = 4096

In [105]:
### analysis parameters
radiusExpanseRatio = 1.5
num_angleSection = 10
num_outsideAngle = 0
num_Xsection = 10
z_offset = 10
analysisParams = [num_angleSection, num_outsideAngle, num_Xsection, z_offset, radiusExpanseRatio]

In [106]:
### import custom functions
import Data_quantification_function_helper as my_help
import Data_quantification_function_intensity_calculation as my_int
import Data_quantification_function_parse_bundle as my_pb
import Data_quantification_functions_plotting as my_plot

In [107]:
"""============== main =============="""

'============== main =============='

In [108]:
sns.set_style("dark")

In [109]:
"""Load data"""
### load summaries
summary_df = pd.read_csv(os.path.join(dataPrefix, summaryFolder, summaryName))
image_list = summary_df.loc[:,'Image_Name'].unique()
ROI_list = summary_df.loc[:,'ROI_Name'].unique()
isExtendedTargetList = False

In [110]:
### 3 or 4 color staining
if(isExtendedTargetList == True):
    targetIndexMatch = {0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:30, 8:20, 9:50, 10:40}
    targetIndexMatch_rev = {0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 30:7, 20:8, 50:9, 40:10}
    ColorCode = {0:'#FFFFFF', 1:'#52FEFE', 2:'#1FF509', 3: '#FF0000', 4: '#CFCF1C', 5: '#FF00FF', 6: '#FFAE01', 20:'#1FF509', 30:'#FF0000', 40:'#CFCF1C', 50:'#FF00FF'}
    channel_mapping = {'RFP':0, 'GFP':1, 'R3':2, 'R4':3, '24B10':4, 0:'RFP', 1:'GFP', 2:'R3', 3:'R4', 4:'24B10'}
    matching_info = (targetIndexMatch, ColorCode, channel_mapping, targetIndexMatch_rev)
else:
    targetIndexMatch = {0:0, 1:2, 2:3, 3:4, 4:5, 5:7}
    targetIndexMatch_rev = {0:0, 2:1, 3:2, 4:3, 5:4, 7:5}
    ColorCode = {1:'#00FFFF', 2:'#1FF509', 3: '#FF0000', 4: '#CFCF1C', 5: '#FF00FF', 6: '#FFAE01', 7:'#FF0000', 0:'#FFFFFF'}
    channel_mapping = {'RFP':0, 'GFP':1, 'R3':2, 'R4':3, 'FasII':4, '24B10':5, 0:'RFP', 1:'GFP', 2:'R3', 3:'R4', 4:'FasII', 5:'24B10'}
    matching_info = (targetIndexMatch, ColorCode, channel_mapping, targetIndexMatch_rev)

In [111]:
image_list

array(['sev-rfpXmDelta-GFP_25C_28hrs_sample3_region1.tif'], dtype=object)

In [112]:
i_image = 0

In [113]:
### load other data
imageName = image_list[i_image]
ROIName = ROI_list[i_image]
# %time image = img_as_float(skiIo.imread(os.path.join(dataPrefix, imageFolder, imageName)))
ROI_df = pd.read_csv(os.path.join(dataPrefix, ROIFolder, ROIName))
ROI_df.rename(columns = {' ':'No'}, inplace = True)
annot_df = summary_df.groupby(['Image_Name']).get_group(image_list[i_image]).reset_index(drop = True)
image_shape = (image.shape[0], image.shape[1], image.shape[2])
# M2P_ratio = (summary_df.iloc[0]['imgX_pixel']/summary_df.iloc[0]['imgX_um'], summary_df.iloc[0]['imgY_pixel']/summary_df.iloc[0]['imgY_um'])

In [114]:
""" Process annotation info"""
bundles_df = my_pb.getBundlesInfo(ROI_df, annot_df, M2P_ratio[0], M2P_ratio[1], isExtendedTargetList)
annot_bundles_df = bundles_df.dropna(axis=0, how='any', inplace = False)
# annot_bundles_df_good = my_pb.Good_QC_df(annot_bundles_df)
# annot_bundles_df_bad = my_pb.Bad_QC_df(annot_bundles_df)

---df_annot---
1
2
3
4
6
9
11
12
13
14
15
16
17
21
22
25
27
28
29
34
35
36
40
38
41
44
45
46
47
50
51
52
56
54
57
90
91
66
68
63
64
69
70
71
72
74
75
76


In [115]:
# """ Process images """
# ### number of channels
# nChannels = image.shape[3]
# if(nChannels == 3):
#     print("3 channels!")
#     ### Seperate channels
#     GFP = image[:,:,:,1]
#     RFP = image[:,:,:,0]
#     Cy5 = image[:,:,:,2]

#     ### normalize GFP & RFP channel
#     image_norm = np.empty(image_shape + (nChannels + 2,), dtype=GFP.dtype, order='C')
    
# #     %time GFP_norm = exposure.equalize_hist(GFP)
#     %time GFP_norm = exposure.rescale_intensity(GFP, in_range = 'image', out_range='dtype')
    
# #     %time RFP_norm = exposure.equalize_hist(RFP)
#     %time RFP_norm = exposure.rescale_intensity(RFP, in_range = 'image', out_range='dtype')
    
#     %time Cy5_norm = exposure.rescale_intensity(Cy5, in_range = 'image', out_range='dtype')
    
#     image_norm[:,:,:,0] = RFP_norm
#     image_norm[:,:,:,1] = GFP_norm
#     image_norm[:,:,:,4] = Cy5_norm

#     R3 = RFP_norm - GFP_norm
#     R3[R3<0] = 0
#     %time R3_norm = exposure.rescale_intensity(R3, in_range = 'image', out_range='dtype')
#     image_norm[:,:,:,2] = R3_norm

#     R4 = RFP_norm * GFP_norm
#     %time R4_norm = exposure.rescale_intensity(R4, in_range = 'image', out_range='dtype')
#     image_norm[:,:,:,3] = R4_norm
    
# elif(nChannels == 4):
#     print("4 channels!")
#     ### Seperate channels
#     GFP = image[:,:,:,1]
#     RFP = image[:,:,:,0]
#     Cy5 = image[:,:,:,2]
#     DAPI = image[:,:,:,3]

#     ### normalize GFP & RFP channel
#     image_norm = np.empty(image_shape + (nChannels + 2,), dtype=GFP.dtype, order='C')
    
# #     %time GFP_norm = exposure.equalize_hist(GFP)
#     %time GFP_norm = exposure.rescale_intensity(GFP, in_range = 'image', out_range='dtype')
    
# #     %time RFP_norm = exposure.equalize_hist(RFP)
#     %time RFP_norm = exposure.rescale_intensity(RFP, in_range = 'image', out_range='dtype')
    
#     %time Cy5_norm = exposure.rescale_intensity(Cy5, in_range = 'image', out_range='dtype')
#     %time DAPI_norm = exposure.rescale_intensity(DAPI, in_range = 'image', out_range='dtype')
    
#     image_norm[:,:,:,0] = RFP_norm
#     image_norm[:,:,:,1] = GFP_norm
#     image_norm[:,:,:,4] = DAPI_norm
#     image_norm[:,:,:,5] = Cy5_norm

#     R3 = RFP_norm - GFP_norm
#     R3[R3<0] = 0
#     %time R3_norm = exposure.rescale_intensity(R3, in_range = 'image', out_range='dtype')
#     image_norm[:,:,:,2] = R3_norm

#     R4 = RFP_norm * GFP_norm
#     %time R4_norm = exposure.rescale_intensity(R4, in_range = 'image', out_range='dtype')
#     image_norm[:,:,:,3] = R4_norm

In [116]:
""" Plot individual bundles """

' Plot individual bundles '

In [117]:
# ind = 1
# bundle_No = list(annot_bundles_df.index)[ind]
# plotSettings = False, False, False, True #isPlotR3Line, isPlotR4Line, isPlotR4s, isLabelOff
# my_plot.plotIndividualBundles(bundle_No, bundles_df, image_norm, M2P_ratio[0], M2P_ratio[1], plotSettings, matching_info)

In [133]:
""" load data """
categoryID = annot_bundles_df.iloc[0]['CategoryID']
sampleID = annot_bundles_df.iloc[0]['SampleID']
regionID = annot_bundles_df.iloc[0]['RegionID']


outputname = 'Control_sample3_region1_v2019422.pickle'
outputDir = os.path.join(dataOutPrefix)
outputDir = os.path.join(outputDir,categoryID)
outputname = os.path.join(outputDir, outputname)

pickle_in = open(outputname,"rb")
outputData= pickle.load(pickle_in)

FileNotFoundError: [Errno 2] No such file or directory: 'Z:\\lab\\Projects\\NSP\\Data_Analysis\\Experiments\\Fate_Switching\\GFP_RFP\\Summaries\\ver2018\\DataOutput_v0422\\Control\\Control_sample3_region1_v2019422.pickle'

In [134]:
bundles_list = annot_bundles_df.index
# ind = list(annot_bundles_df.index).index(bundle_No)

In [128]:
df = pd.DataFrame(index=bundles_list, columns=['R3', 'R4'])

In [132]:
len(outputData['Parameter'])
len(annot_bundles_df.index)

48

In [130]:
for bundle_No in annot_bundles_df.index:
    ind = list(annot_bundles_df.index).index(bundle_No)
    print(ind, bundle_No)
    df.loc[bundle_No, 'R3'] = outputData['Parameter'][ind][0][2][7]
    df.loc[bundle_No, 'R4'] = outputData['Parameter'][ind][0][2][8]

0 1
1 2
2 3
3 4
4 6
5 9
6 11
7 12
8 13
9 14
10 15
11 16
12 17
13 21
14 22
15 25
16 27
17 28
18 29
19 34


IndexError: list index out of range

In [97]:
df.to_csv('Ctrl1.csv')

In [95]:
outputData['Parameter'][ind][0][2][8]

0.2289410149989416

In [69]:
outputData['Parameter'][ind][0][2][7:9]

(0.2669792251144959, 0.2563895904209114)

In [ ]:
gfp = IntensityMatrix[0,1,2,:,:,:,:]
r3 = IntensityMatrix[0,2,2,:,:,:,:]
sum_gfp = np.sum(gfp, axis = 3)
sum_r3 = np.sum(r3, axis = 3)

In [ ]:
def plotSummaryMatrix(matrix, cmap, vmax, tickParams, labelParams):
    xTick_ori, yTick_ori, tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y = tickParams
    ylabel, xlabel, title = labelParams
    xTicks = my_plot.getTickList(tickTypeX, xTick_ori, tickArg2_X)
    yTicks = my_plot.getTickList(tickTypeY, yTick_ori, tickArg2_Y)
    
    ax2 = plt.subplot(111)
    sns.heatmap(matrix, cmap = cmap, yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax)
    ax2.set_ylabel(ylabel, fontsize=16)
    ax2.set_xlabel(xlabel, fontsize=16)
    ax2.set_title(title, fontsize=16)
    #ax2.invert_yaxis()

In [ ]:
vmax_gfp = np.ceil(np.percentile(sum_gfp.flatten(), 90))
vmax_r3 = np.ceil(np.percentile(sum_r3.flatten(), 90))

ylabel = 'Rel. Angle'
xlabel = 'Rel. Length'
labelParams = ylabel, xlabel, ''

tickTypeX = 1
tickTypeY = 2
tickArg2_X = 5
tickArg2_Y = [-1.5, -1, -0.5, 0, 0.5, 1, 1.5]

xTicks_ori = np.round(np.linspace(0, radiusExpanseRatio, sum_gfp[0,:,:].shape[1]), 2)
xTicks = my_plot.getTickList(1, xTicks_ori, tickArg2_X)
yTicks_ori = np.round(params[0][0][1][5], 2)
yTicks = my_plot.getTickList(2, xTicks_ori, tickArg2_Y)

tickParams = xTicks_ori, yTicks_ori, tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y 

# plotSummaryMatrix(np.mean(sum_gfp, axis = 0), 'Greens', vmax_gfp, tickParams, labelParams)
plotSummaryMatrix(np.mean(sum_r3, axis = 0), 'Reds', vmax_r3, tickParams, labelParams)

In [ ]:
plotSummaryMatrix(np.mean(sum_gfp, axis = 0), 'Greens', vmax_gfp, tickParams, labelParams)

In [ ]:
IntensityMatrix = outputData['IntensityMatrix']
params = outputData['Parameter']

In [ ]:
gfp.shape

In [ ]:
sum_gfp.shape

In [ ]:
def plotSummaryMatrix(matrix, cmap, vmax, tickParams, labelParams):
	xTick_ori, yTick_ori, tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y = tickParams
	ylabel, xlabel, title = labelParams
	xTicks = my_plot.getTickList(tickTypeX, xTick_ori, tickArg2_X)
	yTicks = my_plot.getTickList(tickTypeY, yTick_ori, tickArg2_Y)
	
	ax2 = plt.subplot(111)
	sns.heatmap(matrix, cmap = cmap, yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax)
	ax2.set_ylabel(ylabel, fontsize=16)
	ax2.set_xlabel(xlabel, fontsize=16)
	ax2.set_title(title, fontsize=16)
# 	ax2.invert_yaxis()

In [ ]:
vmax_gfp = np.ceil(np.percentile(sum_gfp.flatten(), 90))
vmax_r3 = np.ceil(np.percentile(sum_r3.flatten(), 90))

ylabel = 'Rel. Angle'
xlabel = 'Rel. Length'
labelParams = ylabel, xlabel, 'Title'

tickTypeX = 1
tickTypeY = 2
tickArg2_X = 5
tickArg2_Y = [-1.5, -1, -0.5, 0, 0.5, 1, 1.5]

xTicks_ori = np.round(np.linspace(0, radiusExpanseRatio, sum_gfp[0,:,:].shape[1]), 2)
xTicks = my_plot.getTickList(1, xTick_ori, tickArg2_X)
yTicks_ori = np.round(params[0][0][1][5], 2)
yTicks = my_plot.getTickList(2, yTick_ori, tickArg2_Y)

tickParams = xTicks_ori, yTicks_ori, tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y 

# plotSummaryMatrix(np.mean(sum_gfp, axis = 0), 'Greens', vmax_gfp, tickParams, labelParams)
plotSummaryMatrix(np.mean(sum_r3, axis = 0), 'Reds', vmax_r3, tickParams, labelParams)

In [ ]:
gfp.shape

sum_gfp.shape

def plotSummaryMatrix(matrix, cmap, vmax, tickParams, labelParams):
	xTick_ori, yTick_ori, tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y = tickParams
	ylabel, xlabel, title = labelParams
	xTicks = my_plot.getTickList(tickTypeX, xTick_ori, tickArg2_X)
	yTicks = my_plot.getTickList(tickTypeY, yTick_ori, tickArg2_Y)
	
	ax2 = plt.subplot(111)
	sns.heatmap(matrix, cmap = cmap, yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax)
	ax2.set_ylabel(ylabel, fontsize=16)
	ax2.set_xlabel(xlabel, fontsize=16)
	ax2.set_title(title, fontsize=16)
# 	ax2.invert_yaxis()

vmax_gfp = np.ceil(np.percentile(sum_gfp.flatten(), 90))
vmax_r3 = np.ceil(np.percentile(sum_r3.flatten(), 90))

ylabel = 'Rel. Angle'
xlabel = 'Rel. Length'
labelParams = ylabel, xlabel, 'Title'

tickTypeX = 1
tickTypeY = 2
tickArg2_X = 5
tickArg2_Y = [-1.5, -1, -0.5, 0, 0.5, 1, 1.5]

xTicks_ori = np.round(np.linspace(0, radiusExpanseRatio, sum_gfp[0,:,:].shape[1]), 2)
xTicks = my_plot.getTickList(1, xTick_ori, tickArg2_X)
yTicks_ori = np.round(params[0][0][1][5], 2)
yTicks = my_plot.getTickList(2, yTick_ori, tickArg2_Y)

tickParams = xTicks_ori, yTicks_ori, tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y 

# plotSummaryMatrix(np.mean(sum_gfp, axis = 0), 'Greens', vmax_gfp, tickParams, labelParams)
plotSummaryMatrix(np.mean(sum_r3, axis = 0), 'Reds', vmax_r3, tickParams, labelParams)gfp = IntensityMatrix[0,1,2,:,:,:,:]
r3 = IntensityMatrix[0,2,2,:,:,:,:]
sum_gfp = np.sum(gfp, axis = 3)
sum_r3 = np.sum(r3, axis = 3)

In [ ]:
gfp.shape

In [ ]:
sum_gfp.shape

In [ ]:
def plotSummaryMatrix(matrix, cmap, vmax, tickParams, labelParams):
	xTick_ori, yTick_ori, tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y = tickParams
	ylabel, xlabel, title = labelParams
	xTicks = my_plot.getTickList(tickTypeX, xTick_ori, tickArg2_X)
	yTicks = my_plot.getTickList(tickTypeY, yTick_ori, tickArg2_Y)
	
	ax2 = plt.subplot(111)
	sns.heatmap(matrix, cmap = cmap, yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax)
	ax2.set_ylabel(ylabel, fontsize=16)
	ax2.set_xlabel(xlabel, fontsize=16)
	ax2.set_title(title, fontsize=16)
# 	ax2.invert_yaxis()

In [ ]:
vmax_gfp = np.ceil(np.percentile(sum_gfp.flatten(), 90))
vmax_r3 = np.ceil(np.percentile(sum_r3.flatten(), 90))

ylabel = 'Rel. Angle'
xlabel = 'Rel. Length'
labelParams = ylabel, xlabel, 'Title'

tickTypeX = 1
tickTypeY = 2
tickArg2_X = 5
tickArg2_Y = [-1.5, -1, -0.5, 0, 0.5, 1, 1.5]

xTicks_ori = np.round(np.linspace(0, radiusExpanseRatio, sum_gfp[0,:,:].shape[1]), 2)
xTicks = my_plot.getTickList(1, xTick_ori, tickArg2_X)
yTicks_ori = np.round(params[0][0][1][5], 2)
yTicks = my_plot.getTickList(2, yTick_ori, tickArg2_Y)

tickParams = xTicks_ori, yTicks_ori, tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y 

# plotSummaryMatrix(np.mean(sum_gfp, axis = 0), 'Greens', vmax_gfp, tickParams, labelParams)
plotSummaryMatrix(np.mean(sum_r3, axis = 0), 'Reds', vmax_r3, tickParams, labelParams)

In [ ]:
### thresholds
thr_otsu = np.zeros((4))
thr_li = np.zeros((4))
for channelNo in range(4):
    %time thr_otsu[channelNo] = filters.threshold_otsu(image_norm[:,:,:,channelNo])
    %time thr_li[channelNo] = filters.threshold_li(image_norm[:,:,:,channelNo])

In [ ]:
""" plotting heat maps"""

In [ ]:
# ### plotting
# matrix = IntensityMatrix[sliceTypeNo, :, mind, ind, :,:,:]
# plt.ioff()
# img_name = image_list[i_image]
# tickParams = [1, 2, 5, [-1.5, -1, -0.5, 0, 0.5, 1, 1.5]] ### tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y
# figParams = [mind, params[0][sliceTypeNo][mind], figOutPrefix, img_name, sliceTypeNo, radiusExpanseRatio]
# for thrFunction in [0,1,2]: # different thresholding methods
#     if(thrFunction == 0):
#         thrs = np.zeros((4))
#     elif(thrFunction == 1):
#         thr = thr_otsu
#     elif(thrFunction == 2):
#         thr = thr_li
#     plotOptions = [True, True, True, True, True, thrFunction] ### isPlotLine, isLabelOff, isSave, isTrueXTick, isOriTick, thrFunction            
#     fig = my_plot.plotBundleVsMatrix_all(bundle_No, bundles_df, image_norm, matrix, figParams, tickParams, plotOptions, matching_info)
#     plt.close(fig)

In [ ]:
# for ind in range(len(annot_bundles_df.index)):
# ind = 1
bundle_No = list(annot_bundles_df.index)[ind]
mind = 2 # use T0 as center
sliceTypeNo = 0 # first slicing method
### plotting
matrix = IntensityMatrix[sliceTypeNo, :, mind, ind, :,:,:]
plt.ioff()

img_name = image_list[i_image]
figParams = [mind, params[0][sliceTypeNo][mind], figOutPrefix, img_name, sliceTypeNo, radiusExpanseRatio]
tickParams = [1, 2, 5, [-1.5, -1, -0.5, 0, 0.5, 1, 1.5]] ### tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y
figParams = [mind, params[0][sliceTypeNo][mind], figOutPrefix, img_name, sliceTypeNo, radiusExpanseRatio]
for thrFunction in [0,1,2]: # different thresholding methods
    if(thrFunction == 0):
        thrs = np.zeros((4))
    elif(thrFunction == 1):
        thrs = thr_otsu
    elif(thrFunction == 2):
        thrs = thr_li
    plotOptions = [True, True, True, True, True, thrs, thrFunction] ### isPlotLine, isLabelOff, isSave, isTrueXTick, isOriTick, thrFunction            
    %time fig = my_plot.plotBundleVsMatrix_all(bundle_No, bundles_df, image_norm, matrix, figParams, tickParams, plotOptions, matching_info)
#     plt.close(fig)

In [ ]:
# ### plotting
# mind = 2 # use T0 as center
# sliceTypeNo = 0 # first slicing method
# matrix = IntensityMatrix[sliceTypeNo, :, mind, ind, :,:,:]
# plt.ioff()
# img_name = image_list[i_image]
# figParams = [mind, params[0][sliceTypeNo][mind], figOutPrefix, img_name, sliceTypeNo, radiusExpanseRatio]
# tickParams = [1, 2, 5, [-1.5, -1, -0.5, 0, 0.5, 1, 1.5]] ### tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y
# plotOptions = [True, True, True, True, True, 0] ### isPlotLine, isLabelOff, isSave, isTrueXTick, isOriTick, thrFunction
# fig = my_plot.plotBundleVsMatrix_all(bundle_No, bundles_df, image_norm, matrix, figParams, tickParams, plotOptions, matching_info)
# # plt.close(fig)
# plotOptions = [True, True, True, True, True, 1] ### isPlotLine, isLabelOff, isSave, isTrueXTick, isOriTick, thrFunction 
# fig = my_plot.plotBundleVsMatrix_all(bundle_No, bundles_df, image_norm, matrix, figParams, tickParams, plotOptions, matching_info)
# # plt.close(fig)
# plotOptions = [True, True, True, True, True, 2] ### isPlotLine, isLabelOff, isSave, isTrueXTick, isOriTick, thrFunction 
# fig = my_plot.plotBundleVsMatrix_all(bundle_No, bundles_df, image_norm, matrix, figParams, tickParams, plotOptions, matching_info)
# # plt.close(fig)
# plotOptions = [True, True, True, True, True, 3] ### isPlotLine, isLabelOff, isSave, isTrueXTick, isOriTick, thrFunction 
# fig = my_plot.plotBundleVsMatrix_all(bundle_No, bundles_df, image_norm, matrix, figParams, tickParams, plotOptions, matching_info)
# # plt.close(fig)

In [ ]:
""" get intensity matrix for good bundles"""

In [ ]:
# ### initialization
# print('-----' + image_list[i_image] + '------')
# matrixX = num_angleSection + 2*num_outsideAngle + 1
# matrixY = num_Xsection + 1
# matrixZ = z_offset * 2 + 1
# IntensityMatrix = np.zeros((2, 4, 3, len(annot_bundles_df.index), matrixX, matrixY, matrixZ))
# params = [];

In [ ]:
# ### parse bundles first
# # for ind in range(len(annot_bundles_df.index)):
# ind = 1
# bundle_No = list(annot_bundles_df.index)[ind]
# print("Bundle No: ", bundle_No)
# #     DV = bundles_df.loc[bundle_No, 'Orientation_DV']
# #     AP = bundles_df.loc[bundle_No, 'Orientation_AP']

# ### targets info
# indTs, coordTs = my_plot.getTargetCoords(bundle_No, bundles_df, targetIndexMatch)
# coordR4s = my_plot.getRxCoords(bundle_No, bundles_df, indTs, 4)
# coordR3s = my_plot.getRxCoords(bundle_No, bundles_df, indTs, 3)
# coordRs = np.concatenate((coordR4s, coordR3s))

# ### slice info
# SliceZeroPoint = coordTs[targetIndexMatch_rev[3],:] # T3
# SliceOnePoint = coordTs[targetIndexMatch_rev[30],:] # T7
# # SliceZeroPoint = coordTs[3,:] # T3
# # SliceOnePoint = coordTs[7,:] # T7
# # SliceZeroPoint = coordTs[2,:] # T3
# # SliceOnePoint = coordTs[5,:] # T7


# ## slice radius info
# CutOffPoints = []
# CutOffPoints.append(coordR3s[0,:] + (coordTs[targetIndexMatch_rev[4],:] - coordTs[0,:]) * radiusExpanseRatio) # T4-T0
# CutOffPoints.append(coordR4s[0,:] + (coordTs[targetIndexMatch_rev[4],:] - coordTs[0,:]) * radiusExpanseRatio) # T4-T0
# CutOffPoints.append(coordTs[0,:] + (coordTs[targetIndexMatch_rev[4],:] - coordTs[0,:]) * radiusExpanseRatio) # T4-T0

# CenterPoints = [coordR3s[0,:], coordR4s[0,:], coordTs[0,:]]

# Rcell_nums = [3,4,4]

# ## get slicing params
# pp = [[],[]]
# for mind in range(3):
#     bundleParams = [bundle_No, coordTs, SliceZeroPoint, SliceOnePoint, CutOffPoints[mind], CenterPoints[mind], Rcell_nums[mind]]
#     pp1 = my_int.getSliceParams_v1(analysisParams, bundles_df, bundleParams)
#     pp2 = my_int.getSliceParams_v2(analysisParams, bundles_df, bundleParams, targetIndexMatch_rev)
#     pp[0].append(pp1)
#     pp[1].append(pp2)
# params.append(pp)

# ## calculate matrix
# mind = 2 # use T0 as center
# sliceTypeNo = 0 # first slicing method
# for channelNo in range(4):
#     print(channelNo)
#     IntensityMatrix[sliceTypeNo,channelNo,mind,ind,:,:,:] = my_int.getIntensityMatrix_new(pp[sliceTypeNo][mind], image_norm, channel_mapping[channelNo], channel_mapping)


In [ ]:
# def plotBundleVsMatrix_all(bundle_No, bundles_df, image, intensityMatrix, figParams, tickParams, plotOptions, matching_info):
#     ## unravel parameters
#     mind, params, figurePrefix, img_name, sliceType, radiusExpanseRatio = figParams
#     isPlotLine, isLabelOff, isSave, isTrueXTick, isMask = plotOptions
#     tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y = tickParams
#     targetIndexMatch, ColorCode, channel_mapping, targetIndexMatch_rev = matching_info

#     ## setting seaborn parameters
#     sns.set(font_scale=1.4)
#     sns.set_style("dark")
    
#     Z_offset, num_Xsection, R_Z, phis, CenterPoint, oriYTicks, radius = params
    
#     phi_start = phis[0]
#     phi_end = phis[-1]
                       
#     ### X and Y tick settings
#     if(isTrueXTick):
#         oriXTicks = np.round(np.linspace(0, radiusExpanseRatio, intensityMatrix.shape[2]), 2)
#         print(oriXTicks)
#         xTicks = my_plot.getTickList(tickTypeX, oriXTicks, tickArg2_X)
#     else:
#         xTicks = np.arange(intensityMatrix.shape[1])
#         print(xTicks)
    
#     yTicks = my_plot.getTickList(tickTypeY, oriYTicks, tickArg2_Y)
#     print(oriYTicks)
#     print(yTicks)
    

#     ### R heels info
#     RheelCoords = np.zeros((6,2))
#     RheelCoords[:,0] = list(bundles_df.loc[bundle_No, my_help.group_headers(bundles_df, 'coord_X_R', True)])
#     RheelCoords[:,1] = list(bundles_df.loc[bundle_No, my_help.group_headers(bundles_df, 'coord_Y_R', True)])

#     ### targets info
#     indTs, coordTs = my_plot.getTargetCoords(bundle_No, bundles_df, targetIndexMatch)
#     CoordRs = my_plot.getR4Coords(bundle_No, bundles_df, indTs)        
#     coords = np.concatenate((coordTs, CoordRs))

#     ### image boundary
#     image_bd = (image.shape[2], image.shape[1])
#     xlim = [max(np.floor(coords.min(axis = 0))[0] - 21, 0), min(np.ceil(coords.max(axis = 0))[0] + 21, image_bd[0])]
#     ylim = [max(np.floor(coords.min(axis = 0))[1] - 21, 0), min(np.ceil(coords.max(axis = 0))[1] + 21, image_bd[1])]

#     image_xy = image[R_Z, int(ylim[0]):int(ylim[1]), int(xlim[0]):int(xlim[1]), 0].shape
    

#     ### plot
# #     fig = plt.figure(figsize = (8, 18))
    
#     fig, axes = plt.subplots(4,3)
#     fig.set_size_inches(24,18)
#     fig.suptitle('Bundle No. ' + str(bundle_No), fontsize=18, fontweight='bold')

#     iFig = 0
#     for ax in axes.flat:
#         channelNo = iFig//3
#         thr = filters.threshold_otsu(image[:,:,:,channelNo])
#         if(iFig%3 == 0): ### plot raw figure
#             ### get image crops for plot
#             bundleImage = exposure.rescale_intensity(image[R_Z, int(ylim[0]):int(ylim[1]), int(xlim[0]):int(xlim[1]), channelNo], in_range = 'image', out_range='dtype')
            
#             ## plot original bundle
#             for i in range(6):
#                 ax.plot(RheelCoords[i,0]-xlim[0], RheelCoords[i,1]-ylim[0], color = ColorCode[i+1], marker = 'o', markersize=12)

#             ## plot targets
#             for i in targetIndexMatch.keys():
#                 ax.plot(coordTs[i,0]-xlim[0], coordTs[i,1]-ylim[0],'o', mec = ColorCode[targetIndexMatch[i]], markersize=8, mew = 2.0, mfc = 'none')

# #             ## plot R3/R4s
# #             for i in range(1,len(CoordRs)):
# #                 ax.plot(CoordRs[i,0]-xlim[0], CoordRs[i,1]-ylim[0], color = ColorCode[4], marker = 'o', markersize=7)

#             ## plot lines
#             startPoint = [ CenterPoint[0] + radius * np.cos(phi_start) , CenterPoint[1] + radius * np.sin(phi_start) ]
#             endPoint = [ CenterPoint[0] + radius * np.cos(phi_end) , CenterPoint[1] + radius * np.sin(phi_end) ]
#             # Center-start
#             ax.plot([ CenterPoint[0] - xlim[0], startPoint[0] - xlim[0] ], [ CenterPoint[1] - ylim[0], startPoint[1] - ylim[0] ], '--', alpha = 0.5, color = 'w')
#             # Center-end
#             ax.plot([ CenterPoint[0] - xlim[0], endPoint[0] - xlim[0] ], [ CenterPoint[1] - ylim[0], endPoint[1] - ylim[0] ], '--', alpha = 0.5, color = 'w')            

#             ## plot circle
#             circle = my_plot.drawCircle(CenterPoint, radius)
#             ax.plot(circle[:,0] - xlim[0], circle[:,1] - ylim[0], '--', alpha = 0.5, color = 'w')

#             # plot image
#             ax.imshow(bundleImage, cmap=plt.cm.gray)

#             # labels
#             ax.set_title(channel_mapping[channelNo] + ' channel', fontsize=16)

#             if(isLabelOff):
#                 ax.tick_params(axis='both', which='both', labelbottom='off', labelleft = 'off')
#             else:
#                 ax.tick_params(axis = 'both', labelsize = 12)
                
#         elif(iFig%3 == 1): # plot mean intensity            
#             ### color code settings
#             if((channelNo == 0) | (channelNo == 2)):
#                 colormap = 'Reds'
#             else:
#                 colormap = 'Greens'
                
#             ### matrix and mask
#             matrix1 = np.max(intensityMatrix[channelNo,:,:,:], axis = 2)
#             mask1 = np.zeros_like(matrix1)
#             mask1[matrix1<=thr] = True
            
#             ### plot
#             if(isMask):
#                 with sns.axes_style("dark"):
#                     sns.heatmap(matrix1, cmap = colormap, yticklabels = yTicks, xticklabels = xTicks, ax = ax, mask = mask1)
#             else:
#                 sns.heatmap(matrix1, cmap = colormap, yticklabels = yTicks, xticklabels = xTicks, ax = ax)
#             ax.set_ylabel('Angle', fontsize=16)
#             ax.set_xlabel('length from R-heel', fontsize=16)
#             ax.set_title('Max projection over z-stack', fontsize=16)
#             ax.invert_yaxis()
            
#         elif(iFig%3 == 2): # plot max intensity
#             channelNo = iFig//3
            
#             ### color code settings
#             if((channelNo == 0) | (channelNo == 2)):
#                 colormap = 'Reds'
#             else:
#                 colormap = 'Greens'
            
#             ### matrix and mask
#             matrix2 = np.mean(intensityMatrix[channelNo,:,:,:], axis = 2)
#             mask2 = np.zeros_like(matrix2)
#             mask2[matrix2<=thr] = True
            
#             ### plot
#             if(isMask):
#                 with sns.axes_style("dark"):
#                     sns.heatmap(matrix2, cmap = colormap, yticklabels = yTicks, xticklabels = xTicks, ax = ax, mask = mask2)
#             else:
#                 sns.heatmap(matrix2, cmap = colormap, yticklabels = yTicks, xticklabels = xTicks, ax = ax)            
#             ax.set_ylabel('Angle', fontsize=16)
#             ax.set_xlabel('length from R-heel', fontsize=16)
#             ax.set_title('Mean of z-stack', fontsize=16)
#             ax.invert_yaxis()
        
#         iFig += 1
        
#     plt.subplots_adjust(wspace = 0.2, hspace=0.4)
    
#     if(isSave):
#         plt.ioff()
#         subfolder = 'SliceType' + str(sliceType) + '_' + channel
#         figname = 'Bundle_No_' + str(bundle_No) + '_' + channel + '_' + str(mind)
        
#         check_dir(os.path.join(figurePrefix))
#         check_dir(os.path.join(figurePrefix, img_name))
#         check_dir(os.path.join(figurePrefix, img_name, subfolder))
        
#         plt.savefig(os.path.join(figurePrefix, img_name, subfolder, figname))
    
#     return fig

In [ ]:
# def plotBundleVsMatrix(bundle_No, bundles_df, image, intensityMatrix, figParams, tickParams, plotOptions, matching_info):
#     ## unravel parameters
#     channel, mind, params, figurePrefix, img_name, sliceType, radiusExpanseRatio = figParams
#     isPlotLine, isLabelOff, isSave, isTrueXTick = plotOptions
#     tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y = tickParams
#     targetIndexMatch, ColorCode, channel_mapping, targetIndexMatch_rev = matching_info

#     ## setting seaborn parameters
#     sns.set(font_scale=1.4)
#     sns.set_style("dark")
    
#     Z_offset, num_Xsection, R_Z, phis, CenterPoint, oriYTicks, radius = params
    
#     phi_start = phis[0]
#     phi_end = phis[-1]
                       
#     ### X and Y tick settings
#     if(isTrueXTick):
#         oriXTicks = np.round(np.linspace(0, radiusExpanseRatio, intensityMatrix.shape[1]), 2)
# #         print(oriXTicks)
#         xTicks = my_plot.getTickList(tickTypeX, oriXTicks, tickArg2_X)
#     else:
#         xTicks = np.arange(intensityMatrix.shape[1])
#         print(xTicks)
    
#     yTicks = my_plot.getTickList(tickTypeY, oriYTicks, tickArg2_Y)
#     print(oriYTicks)
#     print(yTicks)
    
#     ### color code settings
#     if(channel == 'GFP'):
#         colormap = 'Greens'
#     elif(channel == 'RFP'):
#         colormap = 'Reds'
#     elif(channel == 'R4'):
#         colormap = 'Greens'
#     elif(channel == 'R3'):
#         colormap = 'Reds'
    

#     ### R heels info
#     RheelCoords = np.zeros((6,2))
#     RheelCoords[:,0] = list(bundles_df.loc[bundle_No, my_help.group_headers(bundles_df, 'coord_X_R', True)])
#     RheelCoords[:,1] = list(bundles_df.loc[bundle_No, my_help.group_headers(bundles_df, 'coord_Y_R', True)])

#     ### targets info
#     indTs, coordTs = my_plot.getTargetCoords(bundle_No, bundles_df, targetIndexMatch)
#     if(channel == 'GFP'):
#         CoordRs = my_plot.getR4Coords(bundle_No, bundles_df, indTs)
#     elif(channel == 'R3'):
#         CoordRs = my_plot.getR3Coords(bundle_No, bundles_df, indTs)
#     elif(channel == 'RFP'):
#         CoordRs = my_plot.getR3Coords(bundle_No, bundles_df, indTs)
#     elif(channel == 'R4'):
#         CoordRs = my_plot.getR4Coords(bundle_No, bundles_df, indTs)
        
#     coords = np.concatenate((coordTs, CoordRs))

#     ### image boundary
#     image_bd = (image.shape[2], image.shape[1])
#     xlim = [max(np.floor(coords.min(axis = 0))[0] - 21, 0), min(np.ceil(coords.max(axis = 0))[0] + 21, image_bd[0])]
#     ylim = [max(np.floor(coords.min(axis = 0))[1] - 21, 0), min(np.ceil(coords.max(axis = 0))[1] + 21, image_bd[1])]

#     image_xy = image[R_Z, int(ylim[0]):int(ylim[1]), int(xlim[0]):int(xlim[1]), 0].shape
    
#     ### get image crops for plot
#     bundleImage = exposure.rescale_intensity(image[R_Z, int(ylim[0]):int(ylim[1]), int(xlim[0]):int(xlim[1]), channel_mapping[channel]], in_range = 'image', out_range='dtype')

#     ### plot
#     fig = plt.figure(figsize = (18,8))
#     fig.suptitle('Bundle No. ' + str(bundle_No) + '; Channel ' + channel, fontsize=18, fontweight='bold')
    
    
#     ### subplot 1: raw image
#     ax1 =  plt.subplot(231)
#     ## plot original bundle
#     for i in range(6):
#         ax1.plot(RheelCoords[i,0]-xlim[0], RheelCoords[i,1]-ylim[0], color = ColorCode[i+1], marker = 'o', markersize=12)

#     ## plot targets
#     for i in targetIndexMatch.keys():
#         ax1.plot(coordTs[i,0]-xlim[0], coordTs[i,1]-ylim[0],'o', mec = ColorCode[targetIndexMatch[i]], markersize=8, mew = 2.0, mfc = 'none')

#     ## plot R3/R4s
#     for i in range(1,len(CoordRs)):
#         ax1.plot(CoordRs[i,0]-xlim[0], CoordRs[i,1]-ylim[0], color = ColorCode[4], marker = 'o', markersize=7)
    
#     ## plot lines
#     startPoint = [ CenterPoint[0] + radius * np.cos(phi_start) , CenterPoint[1] + radius * np.sin(phi_start) ]
#     endPoint = [ CenterPoint[0] + radius * np.cos(phi_end) , CenterPoint[1] + radius * np.sin(phi_end) ]
#     # Center-start
#     ax1.plot([ CenterPoint[0] - xlim[0], startPoint[0] - xlim[0] ], [ CenterPoint[1] - ylim[0], startPoint[1] - ylim[0] ], '--', alpha = 0.5, color = 'w')
#     # Center-end
#     ax1.plot([ CenterPoint[0] - xlim[0], endPoint[0] - xlim[0] ], [ CenterPoint[1] - ylim[0], endPoint[1] - ylim[0] ], '--', alpha = 0.5, color = 'w')            
    
#     ## plot circle
#     circle = my_plot.drawCircle(CenterPoint, radius)
#     ax1.plot(circle[:,0] - xlim[0], circle[:,1] - ylim[0], '--', alpha = 0.5, color = 'w')

#     # plot image
#     ax1.imshow(bundleImage, cmap=plt.cm.gray)
    
#     # labels
#     ax1.set_title(channel + ' channel', fontsize=16)
    
# #     if(channel == 'GFP'):
# #         ax1.set_title('GFP channel', fontsize=16)
# #     elif(channel == 'R3'):
# #         ax1.set_title('RFP - GFP channel', fontsize=16)
# #     elif(channel == 'RFP'):
# #         ax1.set_title('RFP channel', fontsize=16)

#     if(isLabelOff):
#         ax1.tick_params(axis='both', which='both', labelbottom='off', labelleft = 'off')
#     else:
#         ax1.tick_params(axis = 'both', labelsize = 12)


#     ### subfigure 2-4
#     ax2 =  plt.subplot(232)
#     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
#     ax2.set_ylabel('Angle', fontsize=16)
#     ax2.set_xlabel('length from R-heel', fontsize=16)
#     ax2.set_title('Sum over z-stack', fontsize=16)
# #     ax2.invert_yaxis()

#     ax3 =  plt.subplot(233)
#     sns.heatmap(np.mean(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
#     ax3.set_ylabel('Angle', fontsize=16)
#     ax3.set_xlabel('length from R-heel', fontsize=16)
#     ax3.set_title('Mean intensity of z stack', fontsize=16)
#     ax3.invert_yaxis()

#     ax4 =  plt.subplot(235)
#     sns.heatmap(np.median(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
#     ax4.set_ylabel('Angle', fontsize=16)
#     ax4.set_xlabel('length from R-heel', fontsize=16)
#     ax4.set_title('Median intensity of z stack', fontsize=16)
#     ax4.invert_yaxis()
    
#     ax5 =  plt.subplot(236)
#     sns.heatmap(np.max(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
#     ax5.set_ylabel('Angle', fontsize=16)
#     ax5.set_xlabel('length from R-heel', fontsize=16)
#     ax5.set_title('Max intensity of z stack', fontsize=16)
#     ax5.invert_yaxis()
    
#     plt.subplots_adjust(wspace = 0.2, hspace=0.4)
    
#     if(isSave):
#         plt.ioff()
#         subfolder = 'SliceType' + str(sliceType) + '_' + channel
#         figname = 'Bundle_No_' + str(bundle_No) + '_' + channel + '_' + str(mind)
        
#         check_dir(os.path.join(figurePrefix))
#         check_dir(os.path.join(figurePrefix, img_name))
#         check_dir(os.path.join(figurePrefix, img_name, subfolder))
        
#         plt.savefig(os.path.join(figurePrefix, img_name, subfolder, figname))
    
#     return fig

In [ ]:
# ind = 1

# # mind = 2
# # sliceType = 1
# channelNo = 2
# # 
# channel = channel_mapping[channelNo]
# bundle_No = list(annot_bundles_df.index)[ind]

# # plt.ioff()
# img_name = image_list[i_image]
# figParams1 = [channel, mind, pp[sliceTypeNo][mind], figOutPrefix, img_name, sliceTypeNo, radiusExpanseRatio]
# figParams2 = [mind, pp[sliceTypeNo][mind], figOutPrefix, img_name, sliceTypeNo, radiusExpanseRatio]

# tickParams = [1, 2, 5, [-1, -0.5, 0, 0.5, 1]] ### tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y
# plotOptions = [True, True, False, True, False] ### isPlotLine, isLabelOff, isSave, isTrueXTick, isMask
# matrix1 = IntensityMatrix[sliceTypeNo, channelNo, mind, ind, :,:,:]
# matrix2 = IntensityMatrix[sliceTypeNo, :, mind, ind, :,:,:]
# # fig1 = my_plot.plotBundleVsMatrix(bundle_No, bundles_df, image_norm, matrix1, figParams1, tickParams, plotOptions, matching_info)
# fig2 = plotBundleVsMatrix_all(bundle_No, bundles_df, image_norm, matrix2, figParams2, tickParams, plotOptions, matching_info)
# # plt.close(fig)